In [1]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon

<ipython-input-1-7b0e135648b8>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
csv_file = 'info_pozos.csv'
df = pd.read_csv(csv_file)

json_file = 'provincia.json'
with open(json_file) as f:
    data_provincias = json.load(f)

provinces_gdf = gpd.GeoDataFrame.from_features(data_provincias['features'])
provinces_gdf = provinces_gdf.set_crs("EPSG:4326")

def geojson_to_geometry(geojson_str):
    try:
        return shape(json.loads(geojson_str))
    except (json.JSONDecodeError, TypeError, ValueError):
        return None

df['geometry'] = df['geojson'].apply(geojson_to_geometry)
geo_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

result_gdf = gpd.sjoin(geo_gdf, provinces_gdf, how='left', predicate='intersects')

df['provincia_geojson'] = result_gdf['nam']

output_csv_file = 'info_pozos_provincias.csv'
df.to_csv(output_csv_file, index=False)


In [2]:
csv_file = 'info_pozos_provincias.csv'
df = pd.read_csv(csv_file)

def check_provincia(row):
    return row['provincia_geojson'] != row['provincia']

# Apply the comparison function and filter rows where they do not match
df['match_check'] = df.apply(check_provincia, axis=1)

# Filter rows where there is no match (match_check is True)
mismatch_rows = df[df['match_check'] == True]

# Display the mismatched rows
print(mismatch_rows[['provincia', 'provincia_geojson']])

# Optionally, save the result to a new CSV file
output_csv_file = 'provincias_erroneas.csv'
mismatch_rows.to_csv(output_csv_file, index=False)

print(f"Mismatched rows saved to {output_csv_file}")


             provincia provincia_geojson
76               Salta             Jujuy
2792         Rio Negro           Neuquén
2793         Rio Negro           Neuquén
2799         Rio Negro           Neuquén
3181           Mendoza          La Pampa
...                ...               ...
82680  Estado Nacional               NaN
84281  Estado Nacional               NaN
84282              NaN               NaN
84293        Rio Negro         Río Negro
84396  Estado Nacional               NaN

[625 rows x 2 columns]
Mismatched rows saved to provincias_erroneas.csv
